<a href="https://colab.research.google.com/github/hectormelo/Machine-Learning-Techniques/blob/main/Lab_7/Laboratorio_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<img src="https://raw.githubusercontent.com/hectormelo/Machine-Learning-Techniques/main/Banner.png" ><br>
# Machine Learning Techniques - MISIS4219
- Primer semestre - 2024

### Laboratorio 7: Redes Neuronales Recurrentes(RNN)

In [ ]:
!pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=2afd4f65e514d179f67036428b695d5ff5b22c1343737dacbef34f0821ba6082
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from typing import Dict, List, Optional, Sequence, Tuple
import collections
import datetime
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi #librería encargada de procesar los archivos .midi

from IPython import display
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
## Verifiquen donde quedo el dataset en su drive
%cd /content/drive/MyDrive/Colab Notebooks/MLT/Semana 11/Dataset/music_artist

/content/drive/MyDrive/Colab Notebooks/MLT/Semana 11/Dataset/music_artist


In [ ]:
# Creating the required variables

seed = 42
tf.random.set_seed(seed) # para poder recrear el mismo modelo
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

En este caso nuestros datos vienen en formato .mid. Los archivos MIDI no contienen ninguna información de audio real, a diferencia de los formatos .mp3 o .wav. Contienen los datos de audio en forma de qué tipo de notas se reproducirán, la duración, la duración de las notas y el volumen de las notas cuando se reproducen en un software compatible.

In [ ]:
# Colocando el path y cargando los datos

data_dir = pathlib.Path('')

filenames = glob.glob(str(data_dir/'**/*.mid*')) #nos perimite ver la ruta de los archivos que terminan el .mid
print('Number of files:', len(filenames))

Number of files: 292


In [ ]:
# Analizando una de las muestras

sample_file = filenames[1]
print(sample_file)

pm = pretty_midi.PrettyMIDI(sample_file)

print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

# Exatrayendo información de las notas

'''
se usarán tres variables para representar una nota cuando se entrene el modelo: tono,
paso y duración. El tono es la calidad perceptible del sonido como un número de nota MIDI.
El paso es el tiempo transcurrido desde la nota anterior o el inicio de la pista.
La duración es cuánto tiempo se reproducirá la nota en segundos y es la diferencia
entre el final de la nota y el tiempo de inicio de la nota.

'''
for i, note in enumerate(instrument.notes[:10]):
    note_name = pretty_midi.note_number_to_name(note.pitch)
    duration = note.end - note.start
    print(f'{i}: pitch={note.pitch}, note_name={note_name}, duration={duration:.4f}')

brahms/brahms_opus1_2.mid
Number of instruments: 3
Instrument name: Acoustic Grand Piano
0: pitch=60, note_name=C4, duration=1.5036
1: pitch=55, note_name=G3, duration=1.5036
2: pitch=67, note_name=G4, duration=1.4282
3: pitch=60, note_name=C4, duration=1.4282
4: pitch=63, note_name=D#4, duration=0.7517
5: pitch=60, note_name=C4, duration=0.7517
6: pitch=62, note_name=D4, duration=0.6109
7: pitch=59, note_name=B3, duration=0.6109
8: pitch=63, note_name=D#4, duration=0.2350
9: pitch=60, note_name=C4, duration=1.7846


In [ ]:
# Exatraer notas a partir de un archivo .mid
'''
En el fragmento de código a continuación, extraemos estos tres parámetros(pitch, step, duration) que
se utilizarán para construir la red LSTM para calcular y generar música.
'''
def midi_to_notes(midi_file: str):
    pm = pretty_midi.PrettyMIDI(midi_file) #lee el archivo MIDI
    instrument = pm.instruments[0]
    notes = collections.defaultdict(list)

    # ordenando las notas por el tiempo en el que comenzaron
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start

    for note in sorted_notes:
        start = note.start
        end = note.end
        notes['pitch'].append(note.pitch)
        notes['start'].append(start)
        notes['end'].append(end)
        notes['step'].append(start - prev_start)
        notes['duration'].append(end - start)
        prev_start = start

    return pd.DataFrame({name: np.array(value) for name, value in notes.items()}) #se retona un diccionario con los diferentes valores de notas encontrados en el archivo y sus caracteristicas

raw_notes = midi_to_notes(sample_file)
raw_notes.head()

,pitch,start,end,step,duration
0,60,8.917669,10.421230,0.000000,1.503561
1,55,8.917669,10.421230,0.000000,1.503561
2,67,10.500365,11.928579,1.582696,1.428214
3,60,10.500365,11.928579,0.000000,1.428214
4,63,12.003748,12.755440,1.503383,0.751692


In [ ]:
# Conviertiendo en nombre de las notas considerando el valor del tono respectivo

get_note_names = np.vectorize(pretty_midi.note_number_to_name)
sample_note_names = get_note_names(raw_notes['pitch'])
print(sample_note_names[:10])


['C4' 'G3' 'G4' 'C4' 'D#4' 'C4' 'D4' 'B3' 'D#4' 'C4']


In [ ]:
# La siguiente función permite convertir las notas a un archivo .midi
def notes_to_midi(notes: pd.DataFrame, out_file: str, instrument_name: str,
                  velocity: int = 100):

    pm = pretty_midi.PrettyMIDI() #Se crea un objeto PrettyMIDI
    instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

    prev_start = 0
    for i, note in notes.iterrows(): # se recorren las notas
        start = float(prev_start + note['step']) #se asigna la varible star teniendo en cuenta el paso de la nota
        end = float(start + note['duration'])#se asigna la variable end teniendo en cuenta la duracion de la nota

        note = pretty_midi.Note(velocity=velocity, pitch=int(note['pitch']),
                                start=start, end=end) # se crea la nota
        instrument.notes.append(note)
        prev_start = start

    pm.instruments.append(instrument)
    pm.write(out_file)
    return pm

example_file = 'example.midi'
example_pm = notes_to_midi(
    raw_notes, out_file=example_file, instrument_name=instrument_name)

## Desarrollo del dataset para entrenar el modelo

In [ ]:
# creamos el dataset extrayendo las las notas desde los archivos .mid
beethoven= [file_n for file_n in filenames if 'beeth' in file_n] # para este caso entrenaremos con los archivos de Beethoven
all_notes = []
for f in beethoven:
    notes = midi_to_notes(f)
    all_notes.append(notes)

all_notes = pd.concat(all_notes) # Se obtienen todas las notas del archivo en cuestión

n_notes = len(all_notes)
print('Number of notes parsed:', n_notes) # cantidad de notas en nuesto archivo

key_order = ['pitch', 'step', 'duration']
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)

notes_ds = tf.data.Dataset.from_tensor_slices(train_notes) # Documentación: https://www.tensorflow.org/api_docs/python/tf/data/Dataset
notes_ds.element_spec

Number of notes parsed: 53392


TensorSpec(shape=(3,), dtype=tf.float64, name=None)

In [ ]:
'''
El modelo se entrenará en batches de secuencias de notas. Cada ejemplo constará de una
secuencia de notas como características de entrada y la siguiente nota como etiqueta.
De esta forma, el modelo será entrenado para predecir la siguiente nota en una secuencia.
'''
def create_sequences(dataset: tf.data.Dataset, seq_length: int,
                     vocab_size = 128):
    """
    Se retorna un datasets de tensorflow de secuencias y etiquetas
    """
    seq_length = seq_length+1


    windows = dataset.window(seq_length, shift=1, stride=1,
                              drop_remainder=True) #Documentación: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window

    # `flat_map` flattens the" dataset of datasets" into a dataset of tensors
    flatten = lambda x: x.batch(seq_length, drop_remainder=True)
    sequences = windows.flat_map(flatten)

    # se normaliza el tono
    def scale_pitch(x):
        x = x/[vocab_size,1.0,1.0]
        return x

    # separa las etiquetas que en este caso es la próxima nota a predecir
    def split_labels(sequences):
        inputs = sequences[:-1]
        labels_dense = sequences[-1]
        labels = {key:labels_dense[i] for i,key in enumerate(key_order)}

        return scale_pitch(inputs), labels

    return sequences.map(split_labels, num_parallel_calls=tf.data.AUTOTUNE)

'''
Establecemos la longitud de la secuencia para nuestro ejemplo con Beethoven. Se pueden
realizar pruebas con diferentes longitudes (por ejemplo, 40, 120, 160) para ver cuál funciona mejor para los datos.
El tamaño del vocabulario (vocab_size) se establece en 128 y representa todos los tonos admitidos por pretty_midi.
'''

seq_length = 25
vocab_size = 128
seq_ds = create_sequences(notes_ds, seq_length, vocab_size)
seq_ds.element_spec

(TensorSpec(shape=(25, 3), dtype=tf.float64, name=None),
 {'pitch': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'step': TensorSpec(shape=(), dtype=tf.float64, name=None),
  'duration': TensorSpec(shape=(), dtype=tf.float64, name=None)})

In [ ]:
for seq, target in seq_ds.take(1):
  print('sequence shape:', seq.shape)
  print('sequence elements (first 10):', seq[0: 10])
  print()
  print('target:', target)

sequence shape: (25, 3)
sequence elements (first 10): tf.Tensor(
[[0.5625     0.         0.4909455 ]
 [0.5234375  0.         0.4909455 ]
 [0.4921875  0.         0.4909455 ]
 [0.5234375  0.57276975 0.08182425]
 [0.5859375  0.08182425 0.2387265 ]
 [0.5625     0.2387265  0.0795755 ]
 [0.6171875  0.0795755  0.2387265 ]
 [0.5859375  0.2387265  0.0795755 ]
 [0.65625    0.0795755  0.2387265 ]
 [0.6171875  0.2387265  0.08843825]], shape=(10, 3), dtype=float64)

target: {'pitch': <tf.Tensor: shape=(), dtype=float64, numpy=77.0>, 'step': <tf.Tensor: shape=(), dtype=float64, numpy=0.23872650000000029>, 'duration': <tf.Tensor: shape=(), dtype=float64, numpy=0.0795754999999998>}


In [ ]:
batch_size = 64
buffer_size = n_notes - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size) # documentación: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle
            .batch(batch_size, drop_remainder=True)
            .cache() # documentación: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#cache
            .prefetch(tf.data.experimental.AUTOTUNE)) # documentación: https://www.tensorflow.org/api_docs/python/tf/data/Dataset#prefetch

## Crear y entrenar el modelo

El modelo tendrá tres salidas, una para cada variable de nota(tono, paso y duración). Para el paso y la duración, utilizará una función de pérdida personalizada basada en el error cuadrático medio que anima al modelo a generar valores no negativos.

In [ ]:
def mse_with_positive_pressure(y_true: tf.Tensor, y_pred: tf.Tensor):
    mse = (y_true - y_pred) ** 2 # el cuadrado es para no generar valores negativos
    positive_pressure = 10 * tf.maximum(-y_pred, 0.0)
    return tf.reduce_mean(mse + positive_pressure)

In [ ]:
# Developing the model

input_shape = (seq_length, 3)
learning_rate = 0.005
# PAra crear el modelo, en vez de implementar el metodo Sequential, lo haremos capa por capa definiendo lo que entra en a cada capa
inputs = Input(input_shape)
x = LSTM(128)(inputs) # implementamos la LSTM con 128 correspondiente al tamaño del vocavulario (notas)

outputs = {'pitch': Dense(128, name='pitch')(x),
           'step': Dense(1, name='step')(x),
           'duration': Dense(1, name='duration')(x),
          }  # secordar que se hacen tres prediccones: la nota que tiene 128 posibles opciones, el paso que es un valor continuo y la duración que tambíen es contínuo

model = Model(inputs, outputs)

#Como podemos obsevar previamente tenemos tres tipos de salidas diferentes de nuestra red: La nota(pitch), duración(duration) y el paso(step).
#estas tienen características diferentes, mientras que la nota(pitch) es de tipo categórica, la duración y el paso son de tipo continuo, por ende
# deben tener funciones de costo diferentes. Una forma de realizar esto es crear un diccionario donde se llave será el nombre de la salida, el cual debe
# coincidir con el que se colocó en el output de arriba, para saber cual función pertenece a cada salida
loss = {'pitch': SparseCategoricalCrossentropy(from_logits=True),
        'step': mse_with_positive_pressure,
        'duration': mse_with_positive_pressure,
       }

optimizer = Adam(learning_rate=learning_rate)


# Compile y fit al modelo
# Se le agregan diferentes pesos a las perdidas ya que al ejecutar el modelo por primera vez observamos que
# la salida 'pitch' tenía una perdida mayor a las otras dos, por lo que la perdida total recaía sobre esta
# por lo que ajustar el peso para las perdidas compenza ese desbalance.
model.compile(loss = loss,
              loss_weights = {'pitch': 0.05, 'step': 1.0, 'duration':1.0,}, # se le agrega un peso a cada perdida
              optimizer = optimizer)

epochs = 50

history = model.fit(train_ds,
                    epochs=epochs)

model.summary()

Epoch 1/50
833/833 [==============================] - 32s 31ms/step - loss: 1.7073 - duration_loss: 0.0726 - pitch_loss: 3.7510 - step_loss: 1.4472
Epoch 2/50
833/833 [==============================] - 25s 30ms/step - loss: 1.6867 - duration_loss: 0.0688 - pitch_loss: 3.5486 - step_loss: 1.4404
Epoch 3/50
833/833 [==============================] - 25s 30ms/step - loss: 1.6704 - duration_loss: 0.0668 - pitch_loss: 3.3481 - step_loss: 1.4362
Epoch 4/50
833/833 [==============================] - 26s 32ms/step - loss: 1.6597 - duration_loss: 0.0657 - pitch_loss: 3.2195 - step_loss: 1.4330
Epoch 5/50
833/833 [==============================] - 25s 30ms/step - loss: 1.6521 - duration_loss: 0.0641 - pitch_loss: 3.1471 - step_loss: 1.4306
Epoch 6/50
833/833 [==============================] - 25s 30ms/step - loss: 1.6460 - duration_loss: 0.0623 - pitch_loss: 3.1126 - step_loss: 1.4281
Epoch 7/50
833/833 [==============================] - 25s 30ms/step - loss: 1.6406 - duration_loss: 0.0605 - pit

In [ ]:
'''
Para usar el modelo para generar notas, primero deberá proporcionar una secuencia inicial de notas.
La siguiente función genera una nota a partir de una secuencia de notas.
Para el tono de nota, extrae una muestra de la distribución softmax de notas producidas por el modelo,
y no elige simplemente la nota con la probabilidad más alta. Elegir siempre la nota con la mayor
probabilidad daría lugar a la generación de secuencias repetitivas de notas.

El parámetro de temperatura se puede utilizar para controlar la aleatoriedad de las notas generadas.
'''

def predict_next_note(notes: np.ndarray, keras_model: tf.keras.Model,
                      temperature: float = 1.0):
    """Generates a note IDs using a trained sequence model."""

    assert temperature > 0

    # se agrega la dimesión del batch
    inputs = tf.expand_dims(notes, 0) # documentación: https://www.tensorflow.org/api_docs/python/tf/expand_dims

    predictions = model.predict(inputs)
    pitch_logits = predictions['pitch'] # se observan las predicciones del tono
    step = predictions['step'] # predicciones del paso
    duration = predictions['duration'] #predicciones de la duración

    pitch_logits /= temperature
    pitch = tf.random.categorical(pitch_logits, num_samples=1)
    pitch = tf.squeeze(pitch, axis=-1) #Elimina dimensiones de tamaño 1 de un tensor.
    duration = tf.squeeze(duration, axis=-1)
    step = tf.squeeze(step, axis=-1)

    # Los valores de paso y duracion no deben ser negativos
    step = tf.maximum(0, step)
    duration = tf.maximum(0, duration)

    return int(pitch), float(step), float(duration)

## Generación de música

In [ ]:
'''
Con diferentes variables de temperature y diferentes secuencias de inicio, podemos
comenzar a generar música. Hacemos uso de una secuencia de inicio aleatoria con un valor
de temperature aleatorio mediante el cual el modelo LSTM puede continuar construyendo.
Tan pronto como podamos interpretar la siguiente secuencia y tengamos los valores de
tono, paso y duración requeridos, podemos almacenarlos en una lista acumulativa de salidas
generadas. Luego, podemos proceder a eliminar la secuencia de inicio utilizada
anteriormente y hacer uso de la anterior para hacer la siguiente predicción y almacenarla también.
Este paso se puede continuar por un rango hasta que tengamos un tono de
música decente generado por la cantidad de tiempo deseada.

'''
temperature = 2.0
num_predictions = 200 # numero de notas que deseams predecir

sample_notes = np.stack([raw_notes[key] for key in key_order], axis=1)

# normalizamos la secuencia que entra
input_notes = (
    sample_notes[:seq_length] / np.array([vocab_size, 1, 1]))

generated_notes = []
prev_start = 0

for _ in range(num_predictions):
    pitch, step, duration = predict_next_note(input_notes, model, temperature)
    start = prev_start + step
    end = start + duration
    input_note = (pitch, step, duration)
    generated_notes.append((*input_note, start, end))
    input_notes = np.delete(input_notes, 0, axis=0)
    input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)
    prev_start = start

generated_notes = pd.DataFrame(
    generated_notes, columns=(*key_order, 'start', 'end'))

generated_notes.head(10)

1/1 [==============================] - 0s 16ms/step


,pitch,step,duration,start,end
0,74,0.138029,0.536006,0.138029,0.674035
1,79,0.187040,1.237392,0.325069,1.562461
2,88,0.186110,1.234650,0.511179,1.745829
3,85,0.180170,1.242237,0.691349,1.933586
4,88,0.176936,1.248854,0.868286,2.117139
5,100,0.175018,1.273040,1.043304,2.316344
6,86,0.170578,1.266537,1.213882,2.480418
7,84,0.172902,1.268294,1.386784,2.655078
8,82,0.173964,1.286367,1.560747,2.847114
9,87,0.176890,1.280075,1.737637,3.017713


In [ ]:
#almacenar el .midi con las notas que se generaron
out_file = 'output.midi'
out_pm = notes_to_midi(
    generated_notes, out_file=out_file, instrument_name=instrument_name)



Tranformar el archivo .midi en .wav por Python resulta ser tedioso debido a incompatibilidad de versiones en algunas librerías, por ende se deja una página web para hacerlo.
En el siguiente enlace se puede realizar la converison de midi a wav: https://audio.online-convert.com/es/convertir/midi-a-wav

In [ ]:
#resultado de la predicción
display.Audio(filename='../output.wav', rate=16000)

## El resultado suena un poco básico, ¿Que estrategias usarías para mejorarlo?